In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Pakistan COVID19 Data provided till June-2020

In [ ]:
data = pd.read_csv("/kaggle/input/pakistan-corona-virus-citywise-data/PK COVID-19-3jun.csv")
coords = pd.read_csv("/kaggle/input/pakistan-corona-virus-citywise-data/covid_cases_city_coordinates.csv")

In [ ]:
print("Total Number of Rows: ", len(data))
data.dtypes                                      #Data Type of the Data Columns

In [ ]:
data['Date'] =  pd.to_datetime(data['Date'], infer_datetime_format=True)
data.dtypes                                      #Data Type of the Data Columns

In [ ]:
data.tail()

### **City Coordinates Data**

In [ ]:
#Checking Null OR Empty Data Cell
coords.isnull().sum()

In [ ]:
# Rename the Name of Columns for easy to use and get Null value of lng and lat rows
coords = coords.rename(columns={'City':'city', 'Latitude (generated)':'lat', 'Longitude (generated)':'lng'})
coords[coords.lat.isnull()]

In [ ]:
#Try to reducing the Null/Empty values insert lng and lat of the following cities

coords.loc[coords["city"] == 'Chitral', ['lng']] = 71.7889
coords.loc[coords["city"] == 'Chitral', ['lat']] = 35.8511 
                                         
coords.loc[coords["city"] == 'Chitral Lower', ['lng']] = 71.7889
coords.loc[coords["city"] == 'Chitral Lower', ['lat']] = 35.8511

coords.loc[coords["city"] == 'Chitral Upper', ['lng']] = 71.7889
coords.loc[coords["city"] == 'Chitral Upper', ['lat']] = 35.8511

coords.loc[coords["city"] == 'Jamshoro', ['lng']] = 68.2822
coords.loc[coords["city"] == 'Jamshoro', ['lat']] = 25.4283

coords.loc[coords["city"] == 'Khanewal', ['lng']] = 71.9321
coords.loc[coords["city"] == 'Khanewal', ['lat']] = 30.3017

coords[coords.lat.isnull()]

### Our Actual working Data

In [ ]:
data.head()

## Identifies the Repetition/Duplication

In [ ]:
data["Province"].value_counts()

### Try to Remove Repetition/Duplication

In [ ]:
spelling_correction = {
    "khyber Pakhtunkhwa": "Khyber Pakhtunkhwa",
    "islamabad Capital Territory": "Islamabad Capital Territory"
}
data["Province"] = data["Province"].replace(spelling_correction)
data["Province"].value_counts()

In [ ]:
sns.swarmplot(x="Cases", y="Province", data=data,)

In [ ]:
corr = data.corr()
sns.heatmap(corr,annot=True)

In [ ]:
data_province = data.groupby('Province', axis=0).sum()
data_province

In [ ]:
province_sorted = data_province.sort_values(by='Province', ascending=False)

In [ ]:
province_sorted['Cases'].plot(kind='pie',
                            figsize = (18, 8),
                            autopct = '%1.1f%%',
                            startangle = 0,
                            labels = None,
                            textprops = dict(color='black', fontsize=16),
                            explode = [0, 0.02, 0.05, 0.3, 0, 0.3, 0.2, 0.1]
                         )
plt.title('Reported Cases Province Wise', fontsize=20)
plt.axis('equal')
plt.legend(labels=province_sorted.index, loc='upper right', fontsize=14)
plt.show()

In [ ]:
province_sorted['Recovered'].plot(kind='pie',
                                  figsize = (18, 8),
                                  autopct = '%1.1f%%', startangle = 0, labels = None,
                                  textprops = dict(color='black', fontsize=16),
                                  explode = [0, 0.02, 0.05, 0.3, 0, 0.3, 0.2, 0.1]
                         )
plt.title('Reported Recovered Province Wise', fontsize=20)
plt.axis('equal')
plt.legend(labels=province_sorted.index, loc='upper right', fontsize=14)
plt.show()

In [ ]:
province_sorted['Deaths'].plot(kind='pie',
                            figsize = (18, 8),
                            autopct = '%1.1f%%',
                            startangle = 0,
                            labels = None,
                            textprops = dict(color='w', fontsize=16),
                            explode = [0, 0.02, 0.05, 0.3, 0, 0.3, 0.2, 0.1]
                         )
plt.title('Reported Deaths Province Wise', fontsize=20)
plt.axis('equal')
plt.legend(labels=province_sorted.index, loc='upper right', fontsize=14)
plt.show()

In [ ]:
province_sorted.plot(kind='area', stacked=True,
                     figsize=(18, 8),
                     )
plt.title('Provinces wise COVID19 Cases, Deaths & Recovered Trend', fontsize=20)
plt.xlabel('\n\nPakistan Provinces', fontsize=20)
plt.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'], y=data['Cases'],
                    mode='lines+markers',
                    name='Cases'))
fig.add_trace(go.Scatter(x=data['Date'], y=data['Deaths'],
                    mode='lines+markers',
                    name='Deaths'))
fig.add_trace(go.Scatter(x=data['Date'], y=data['Recovered'],
                    mode='lines+markers',
                    name='Recoveries'))
fig.show()

In [ ]:
px.scatter(data, x= 'Date', y = 'Cases', trendline = "ols")

# Geographically Visualization of COVID19 Data

In [ ]:
data.head()

In [ ]:
coords.head()

In [ ]:
cities = list(set(data['City'].values))
#data.dropna()
data_map = pd.DataFrame(columns=['Province', 'City', 'Cases', 'Recovered', 'Deaths', 'Longitude', 'Latitude'])

for city in cities:
    cases =0
    deaths =0
    recovered =0
    for i,row in data.iterrows():
        if row['City'] == city:
            province = row['Province']
            cases += row['Cases']
            deaths += row['Deaths']
            recovered += row['Recovered']
            
    lat = coords[coords['city'] == city]['lat']
    lng = coords[coords['city'] == city]['lng']
    df = { "City": city, "Province": province, "Cases": cases, "Recovered": recovered, "Deaths": deaths, "Longitude": lng, "Latitude": lat}
    data_map = data_map.append(pd.DataFrame(df))
print("Number of Rows in New DataFrame: ", len(data_map))

In [ ]:
data_map.isna().sum()

In [ ]:
data_map = data_map.dropna()
data_map.isna().sum()

In [ ]:
data_map.head()

In [ ]:
#Calculating Total Number of Reported Cases, Recovered and Deaths
totalCases = 0
totalRecovered = 0
totalDeaths = 0
for i, row in data_map.iterrows():
    totalCases += row['Cases']
    totalRecovered += row['Recovered']
    totalDeaths += row['Deaths']

print("Total Number of Reported Cases: " + str(totalCases) + "\nTotal Number of Recovered: " + str(totalRecovered) + 
      "\nTotal Number of Deaths: " + str(totalDeaths))

In [ ]:
data_map.columns

In [ ]:
pakistan_map = folium.Map(location = [30.37,69.34],
                          zoom_start= 5.5,
                          min_zoom = 5.5,
                          max_zoom = 8,
                          tiles='OpenStreetMap' )

for city, prov, cases, recover, deaths, lng, lat in zip(list(data_map['City']), 
                                                        list(data_map['Province']), 
                                                        list(data_map['Cases']), 
                                                        list(data_map['Recovered']), 
                                                        list(data_map['Deaths']), 
                                                        list(data_map['Longitude']), 
                                                        list(data_map['Latitude'])):
    folium.CircleMarker(location=[lat, lng], radius=(float(cases)/totalCases)*100., 
                        color="red", stroke=True, weight=1, opacity=0.8, 
                        fill=True, fill_color= "red", fill_opacity = 0.5,
                        tooltip = "<strong>"+city+"</strong><br>Cases:"+str(cases)+
                                       "<br>Recovered:"+str(recover)+
                                       "<br>Deaths:"+str(deaths)
                       ).add_to(pakistan_map )
    
pakistan_map

In [ ]:
pakistan_map = folium.Map(location = [30.37,69.34],
                          zoom_start= 5.5,
                          min_zoom = 5.5,
                          max_zoom = 8,
                          tiles='OpenStreetMap' )

for city, prov, cases, recover, deaths, lng, lat in zip(list(data_map['City']), 
                                                        list(data_map['Province']), 
                                                        list(data_map['Cases']), 
                                                        list(data_map['Recovered']), 
                                                        list(data_map['Deaths']), 
                                                        list(data_map['Longitude']), 
                                                        list(data_map['Latitude'])):
    
    folium.CircleMarker(location=[lat, lng], radius=(float(deaths)/totalDeaths)*100, 
                        color="red",  stroke=True, weight=1, opacity=0.8, 
                        fill=False, 
                        tooltip = "<strong>"+city+"</strong><br>Cases:"+str(cases)+
                                       "<br>Recovered:"+str(recover)+
                                       "<br>Deaths:"+str(deaths)).add_to(pakistan_map )
    
    folium.CircleMarker(location=[lat, lng], radius=(float(cases)/totalCases)*100, 
                        color="purple",  stroke=True, weight=1, opacity=0.8, 
                        fill=True, fill_color= "purple",
                        tooltip = "<strong>"+city+"</strong><br>Cases:"+str(cases)+
                                       "<br>Recovered:"+str(recover)+
                                       "<br>Deaths:"+str(deaths)).add_to(pakistan_map )
    
    folium.CircleMarker(location=[lat, lng], radius=(float(recover)/totalRecovered)*100, 
                        color="green",  stroke=True, weight=1, opacity=0.8, 
                        fill=True, fill_color= "green", fill_opacity = 0.2,
                        tooltip = "<strong>"+city+"</strong><br>Cases:"+str(cases)+
                                       "<br>Recovered:"+str(recover)+
                                       "<br>Deaths:"+str(deaths)).add_to(pakistan_map )
    

pakistan_map

Hope attempt is appriciated with lots of love, looking for your guidance like, **UPVOTES** for motivation Previously Notebook comments and like/Upvores highly appriciated and motivated Heartly Thanks with lots of **LOVE**
